In [2]:
import polars as pl
import numpy as np

Сопоставление размеров в разных проекциях.


In [4]:
df = pl.read_csv("DF_semifinal.csv")
df.head(3)

id,name,comment,axial.left.anterior,axial.left.posterior,axial.left.medial,axial.left.lateral,axial.right.anterior,axial.right.posterior,axial.right.medial,axial.right.lateral,sagittal.left.anterior,sagittal.left.superior,sagittal.left.posterior,sagittal.right.anterior,sagittal.right.superior,sagittal.right.posterior,coronal.left.lateral,coronal.left.medial,coronal.left.superior,coronal.right.lateral,coronal.right.medial,coronal.right.superior,issue.lateral.left,issue.lateral.right,issue.medial.left,issue.medial.right,issue.compression.left,issue.compression.right,issue.distraction.left,issue.distraction.right,issue.distal.left,issue.distal.right,issue.mesial.left,issue.mesial.right
i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""Останькович В.…",null,-1.0,-1.0,3.1,1.6,null,null,null,null,2.3,2.9,1.6,2.3,3.0,1.4,1.6,2.6,2.7,1.7,5.1,2.6,1.0,1.0,null,null,null,null,null,null,1.0,1.0,null,null
2,"""Самойлова Е.Г.…",null,-1.0,-1.0,1.7,2.3,-1.0,-1.0,2.0,2.3,2.2,2.1,1.6,2.8,2.4,1.9,2.2,2.4,1.9,1.9,1.9,2.3,null,null,1.0,null,null,null,null,null,1.0,1.0,null,null
3,"""Авакимова М.Н.…",null,-1.0,-1.0,8.4,2.4,null,null,null,null,2.9,4.2,3.7,3.6,4.2,2.7,3.2,6.3,4.2,4.1,6.3,3.0,null,1.0,null,null,null,null,null,null,null,1.0,null,null


In [5]:
df = df.fill_null(0)
df.head(3)

id,name,comment,axial.left.anterior,axial.left.posterior,axial.left.medial,axial.left.lateral,axial.right.anterior,axial.right.posterior,axial.right.medial,axial.right.lateral,sagittal.left.anterior,sagittal.left.superior,sagittal.left.posterior,sagittal.right.anterior,sagittal.right.superior,sagittal.right.posterior,coronal.left.lateral,coronal.left.medial,coronal.left.superior,coronal.right.lateral,coronal.right.medial,coronal.right.superior,issue.lateral.left,issue.lateral.right,issue.medial.left,issue.medial.right,issue.compression.left,issue.compression.right,issue.distraction.left,issue.distraction.right,issue.distal.left,issue.distal.right,issue.mesial.left,issue.mesial.right
i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""Останькович В.…",null,-1.0,-1.0,3.1,1.6,0.0,0.0,0.0,0.0,2.3,2.9,1.6,2.3,3.0,1.4,1.6,2.6,2.7,1.7,5.1,2.6,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,"""Самойлова Е.Г.…",null,-1.0,-1.0,1.7,2.3,-1.0,-1.0,2.0,2.3,2.2,2.1,1.6,2.8,2.4,1.9,2.2,2.4,1.9,1.9,1.9,2.3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,"""Авакимова М.Н.…",null,-1.0,-1.0,8.4,2.4,0.0,0.0,0.0,0.0,2.9,4.2,3.7,3.6,4.2,2.7,3.2,6.3,4.2,4.1,6.3,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
excluded = [0, -1]

In [19]:
df_left_superior = df.select(["id", "^.*left.superior$"]).with_columns(
    pl.when(
        pl.col("coronal.left.superior").is_in(excluded)
        | pl.col("sagittal.left.superior").is_in(excluded)
    )
    .then(0)
    .otherwise(pl.col("sagittal.left.superior") - pl.col("coronal.left.superior"))
    .alias("diff"),
)


df_left_superior.describe()

describe,id,sagittal.left.superior,coronal.left.superior,diff
str,f64,f64,f64,f64
"""count""",379.0,379.0,379.0,379.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",190.583113,2.223615,1.705145,0.296385
"""std""",110.162268,1.647448,1.565731,0.620269
"""min""",1.0,-1.0,-1.0,-1.5
"""25%""",96.0,0.9,0.0,0.0
"""50%""",190.0,2.3,1.9,0.0
"""75%""",286.0,3.2,2.7,0.5
"""max""",381.0,7.6,7.3,3.63


In [21]:
df_left_superior.filter(pl.col("diff") > 0.5)

id,sagittal.left.superior,coronal.left.superior,diff
i64,f64,f64,f64
5,4.1,3.5,0.6
9,3.0,2.4,0.6
11,3.2,2.6,0.6
12,2.8,2.1,0.7
14,3.5,2.1,1.4
17,3.1,1.8,1.3
23,4.6,2.9,1.7
25,3.6,2.7,0.9
26,3.2,2.0,1.2
